# 네이버 지식인 페이지 정적 크롤링
정적 크롤링을 통해 네이버 지식인 페이지에서 데이터를 추출하는 방법을 학습합니다.

### 1. 필요한 라이브러리 설치 및 임포트
먼저, 웹 크롤링을 위해 필요한 라이브러리들을 설치하고 임포트합니다.

- bs4: BeautifulSoup 라이브러리는 HTML/XML 페이지를 파싱하여 데이터를 쉽게 추출할 수 있게 도와줍니다.
- requests: HTTP 요청을 보내 웹 페이지의 HTML을 받아오는 라이브러리입니다.
- pandas: 데이터를 표 형태로 처리하고, 엑셀 파일로 저장하는 데 사용됩니다.
- openpyxl: pandas가 엑셀 파일을 처리할 수 있도록 도와주는 라이브러리입니다.

In [1]:
!pip install bs4
!pip install requests
!pip install pandas
!pip install openpyxl

### 2. HTML 페이지 불러오기 및 파싱
이제 웹 페이지를 불러와서 HTML을 파싱하여 필요한 데이터를 추출하는 작업을 시작합니다.

- requests.get(url): 지정한 URL에 HTTP GET 요청을 보냅니다.
- BeautifulSoup(html, 'html.parser'): 응답 받은 HTML을 BeautifulSoup을 사용해 파싱합니다.

In [1]:
from bs4 import BeautifulSoup
import requests

# 네이버 지식인 삼성전자 검색 페이지 URL
url = "https://kin.naver.com/search/list.naver?query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90"
response = requests.get(url)  # 요청 보내기
html = response.text  # 응답 받은 HTML 문서
soup = BeautifulSoup(html,'html.parser')  # BeautifulSoup으로 파싱
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta contents="always" name="referrer"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="삼성전자의 지식iN Q&amp;A 검색결과입니다. 궁금증을 해결하지 못했다면 지식iN '질문하기'를 해보세요." name="description">
<meta content="width=1024" name="viewport"/>
<meta content="none" name="msapplication-config">
<link href="https://ssl.pstatic.net/static.kin/static/pc/20250908150228/css/min/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20250908150228/css/min/components.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20250908150228/css/min/other.css" rel="stylesheet" type="text/css"/>
<script>
	
	var pcDomain = "kin.naver.com";
	var pcDomainWithProtocol = "https://kin.naver.com";
	var pcKinServiceProtocol = "https";

	var mobileDomain = "m.kin.naver.com";
	var mobileDomainWithProtocol = "https://m.kin.naver.com";
	var mobileKinServiceProtocol = "https";

	var uplo

### 3. 특정 HTML 요소 선택
크롤링할 HTML 요소를 선택하기 위해 CSS 선택자를 사용하여 데이터를 추출합니다.

- soup.select_one(): CSS 선택자를 사용하여 첫 번째 일치하는 요소를 선택합니다.
- tree: 선택된 HTML 요소(첫 번째 질문)에 대한 정보를 담고 있습니다.

In [7]:
# 첫 번째 질문 요소 선택
tree = soup.select_one('.basic1 > li > dl')
tree  # 첫 번째 질문의 HTML 구조를 출력하여 확인

<dl>
<dt>
<a class="_nclicks:kin.txt _searchListTitleAnchor" href="https://kin.naver.com/qna/detail.naver?d1id=4&amp;dirId=40102&amp;docId=488903238&amp;qb=7IK87ISx7KCE7J6Q&amp;enc=utf8" target="_blank"><b>삼성전자</b>주식</a>
<img alt="지식파트너 답변" class="ico_pro" height="14" src="https://ssl.pstatic.net/static/kin/09renewal/ico_kin_partner.gif" width="51"/>
</dt>
<dd class="txt_inline">2025.09.17.</dd>
<dd><b>삼성전자</b> 주식이 앞으로 오를까요? 안녕하세요?... <b>삼성전자</b> 주가 상승 여부는 현재로서는 섣불리 판단하기... ◆ 긍정적인 요인 ㅇAI 반도체 및 HBM 사업 성장: <b>삼성전자</b>는... </dd>
<dd class="tag_area">
</dd>
<dd class="txt_block">
<a class="txt_g1 _nclicks:kin.cat1" href="/search/list.naver?query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&amp;section=qna">Q&amp;A</a>
								
									 &gt; <a class="txt_g1 _nclicks:kin.cat2" href="/search/list.naver?query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&amp;section=qna&amp;dirId=40102">주식, 증권</a>
<span class="bar">|</span>
<span class="hit">답변수 10</span> UP 23
							
							
							</dd>
</dl>

### 4. 정보 추출: 제목, 링크, 날짜, 카테고리, 답변수
선택한 HTML 요소에서 원하는 데이터를 추출합니다.

- title_tag.text: title_tag 요소에서 텍스트(제목)를 추출합니다.
- title_tag.attrs['href']: title_tag 요소에서 링크를 추출합니다.
- date_tag.text, category_tag.text: 각각 작성일과 카테고리를 추출합니다.
- hit_tag.text.split(): 답변수를 추출하고 불필요한 문자를 제거합니다.

In [18]:
# 제목과 링크 추출
title_tag = tree.select_one("._nclicks\\:kin\\.txt._searchListTitleAnchor")
title = title_tag.text
link = title_tag.attrs['href'] 
print(title, link)


삼성전자에 벤큐 모니터 파나요 https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1010401&docId=488053141&qb=7IK87ISx7KCE7J6Q&enc=utf8


In [ ]:
# 날짜 추출
date_tag = tree.select_one(".txt_inline")
date = date_tag.text
print(date)


2025.08.25.


In [32]:
# 카테고리 추출
category_tag = tree.select_one(".txt_g1._nclicks\\:kin\\.cat2")
category = category_tag.text
print(category)


모니터


In [9]:
# 조회수 추출
hit_tag = tree.select_one('.hit')
texts = hit_tag.text
hit = texts.split()[1]
print(hit)


10


### 5. 한 페이지에서 모든 질문 정보 추출
한 페이지에 여러 질문이 있을 때, 모든 질문의 정보를 추출합니다.

- soup.select(): 여러 개의 요소를 선택하여 리스트로 반환합니다.
- 각 질문에 대해 for 루프를 돌며 제목, 링크, 날짜, 카테고리, 조회수를 추출합니다.

In [38]:
# 여러 질문 정보 추출
trees = soup.select(".basic1 > li > dl")
for tree in trees:
    title = tree.select_one("._nclicks\\:kin\\.txt").text
    link = tree.select_one("._nclicks\\:kin\\.txt").attrs['href']
    date = tree.select_one(".txt_inline").text
    category = tree.select_one("._nclicks\\:kin\\.cat2").text
    hit = tree.select_one(".hit").text.split()[1]
    
    # 출력
    print(title, link, date, category, hit)

삼성전자에 벤큐 모니터 파나요 https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1010401&docId=488053141&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.08.25. 모니터 5
삼성전자 주식 https://kin.naver.com/qna/detail.naver?d1id=4&dirId=40102&docId=487572786&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.07.30. 주식, 증권 19
삼성전자 DS부문 면접 준비에 필요한 팁은? https://kin.naver.com/qna/detail.naver?d1id=4&dirId=40609&docId=486659911&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.08.27. 생산, 기술 3
삼성전자 로고 있는 마우스패드 https://kin.naver.com/qna/detail.naver?d1id=5&dirId=5020402&docId=488543373&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.09.03. 키보드, 마우스 4
삼성전자 냉장고 소음 https://kin.naver.com/qna/detail.naver?d1id=6&dirId=60213&docId=488077668&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.08.19. 손해배상 1
쿠팡 삼성 전자 제품 https://kin.naver.com/qna/detail.naver?d1id=5&dirId=501&docId=488177860&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.08.21. 가전제품 3
삼성전자 스탠드 에어컨 https://kin.naver.com/qna/detail.naver?d1id=5&dirId=50104&docId=488562606&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.09.03. 청소, 주방, 계절 가전 4
삼성전자 5급 채용 https://kin.

### 6. 여러 페이지 크롤링
페이지를 변경하면서 여러 페이지의 데이터를 크롤링합니다.

- for page_num in range(1, 4): 1페이지부터 3페이지까지 순차적으로 크롤링합니다.
- 각 페이지에서 데이터를 추출하여 data 리스트에 추가하고, 이를 pandas DataFrame으로 변환하여 엑셀 파일로 저장합니다.

In [43]:
# 여러 페이지에서 정보 추출
data = []
for page_num in range(1, 4):  # 1~3페이지 크롤링
    url = f"https://kin.naver.com/search/list.naver?query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page={page_num}"
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    trees = soup.select(".basic1 > li > dl")
    
    for tree in trees:
        ################
        title = tree.select_one("._nclicks\\:kin\\.txt").text
        link = tree.select_one("._nclicks\\:kin\\.txt").attrs['href']
        date = tree.select_one(".txt_inline").text
        category = tree.select_one("._nclicks\\:kin\\.cat2").text
        hit = tree.select_one(".hit").text.split()[1]
        # 데이터를 리스트에 추가
        data.append([title, link, date, category, hit])

# DataFrame으로 변환
import pandas as pd
df = pd.DataFrame(data,columns=['title','link','date','category','hit'])

df


,title,link,date,category,hit
0,삼성전자에 벤큐 모니터 파나요,https://kin.naver.com/qna/detail.naver?d1id=1&...,2025.08.25.,모니터,5
1,삼성전자 주식,https://kin.naver.com/qna/detail.naver?d1id=4&...,2025.07.30.,"주식, 증권",19
2,삼성전자 DS부문 면접 준비에 필요한 팁은?,https://kin.naver.com/qna/detail.naver?d1id=4&...,2025.08.27.,"생산, 기술",3
3,삼성전자 로고 있는 마우스패드,https://kin.naver.com/qna/detail.naver?d1id=5&...,2025.09.03.,"키보드, 마우스",4
4,삼성전자 냉장고 소음,https://kin.naver.com/qna/detail.naver?d1id=6&...,2025.08.19.,손해배상,1
5,쿠팡 삼성 전자 제품,https://kin.naver.com/qna/detail.naver?d1id=5&...,2025.08.21.,가전제품,3
6,삼성전자 스탠드 에어컨,https://kin.naver.com/qna/detail.naver?d1id=5&...,2025.09.03.,"청소, 주방, 계절 가전",4
7,삼성전자 5급 채용,https://kin.naver.com/qna/detail.naver?d1id=4&...,2024.11.13.,"생산, 기술",3
8,삼성전자 상간녀소송중인데요. 상간녀가,https://kin.naver.com/qna/detail.naver?d1id=6&...,2025.05.27.,"가족, 이혼",3
9,삼성전자 에어컨 청소,https://kin.naver.com/qna/detail.naver?d1id=5&...,2025.08.11.,"청소, 주방, 계절 가전",9


### 7. 결과 저장
위의 크롤링한 데이터를 엑셀 파일로 저장합니다.

- df.to_excel(): 추출한 데이터를 엑셀 파일로 저장합니다. index=False를 설정하여 인덱스를 제외하고 저장합니다.

In [44]:
# pandas를 사용해 엑셀로 저장

df.to_excel('jisigin.xlsx',index=False)
